In [38]:
!pip install pulp #Python Linear Programming
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK
import pandas as pd
import numpy as np

from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/Modelling")

df = pd.read_csv("Modelling data.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
df.head()

,X1,X2,X3,X4,Y1,Y2
0,114.8,0.75,3,16660,8.71,0.383142
1,106.8,0.70,3,12376,8.10,0.016181
2,84.6,0.86,4,24463,6.20,0.041408
3,77.7,0.63,4,6810,3.12,0.031153
4,56.3,0.81,2,42442,0.12,0.071429


In [40]:
df.dtypes

X1    float64
X2    float64
X3      int64
X4      int64
Y1    float64
Y2    float64
dtype: object

In [41]:
weights_matrix = np.empty((6,4))

In [48]:
weights_matrix = np.empty((9,9))
for i in range(9):
    model = pulp.LpProblem("Investment Analysis", pulp.LpMaximize) # Create an LP maximization problem
    m1 = pulp.LpVariable("m1", lowBound=0, upBound=None, cat='Continuous') # Create a variable m1>=0
    m2 = pulp.LpVariable("m2", lowBound=0, upBound=None, cat='Continuous') # Create a variable m2>=0
    n1 = pulp.LpVariable("n1", lowBound=0, upBound=None, cat='Continuous') # Create a variable n1>=0
    n2 = pulp.LpVariable("n2", lowBound=0, upBound=None, cat='Continuous') # Create a variable n2>=0
    n3 = pulp.LpVariable("n3", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0
    n4 = pulp.LpVariable("n4", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0


    model += m1*df["Y1"][i] + m2*df["Y2"][i]  #objective function

    model += n1*df["X1"][i] + n2*df["X2"][i] + n3*df["X3"][i]+ n4*df["X4"][i] == 1 #first constraint (equality)

    for j in range(9):
        model += m1*df["Y1"][j] + m2*df["Y2"][j] - n1*df["X1"][j] - n2*df["X2"][j] - n3*df["X3"][j] - n4*df["X4"][j] <= 0

    status = model.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt']))
    print("Model Status:{}".format(pulp.LpStatus[model.status]))




    # Displaying Optimal Decision Variables & Reduced Cost per Variable
    list_weights=[]
    for k in model.variables():
	      print(k.name, "=", k.varValue, "\tReduced Cost =", k.dj)
    for k in model.variables():
        list_weights=list_weights+[k.varValue]
    for l in range(6):
        value= list_weights[l]
        weights_matrix[l,i] = value

    # Displaying Optimal Value of Objective Function
    print("Objective=", pulp.value(model.objective))

    # Displaying Shadow Price & Slack Value per Constraint
    print ("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
    for name, m in model.constraints.items():
	      print(name, ":", m, "\t", m.pi, "\t\t", m.slack)

Model Status:Optimal
m1 = 0.0 	Reduced Cost = None
m2 = 2.61 	Reduced Cost = None
n1 = 0.0 	Reduced Cost = None
n2 = 0.0 	Reduced Cost = None
n3 = 0.0 	Reduced Cost = None
n4 = 6.0024e-05 	Reduced Cost = None
Objective= 0.9999999988199999

Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : 114.8*n1 + 0.75*n2 + 3*n3 + 16660*n4 = 1.0 	 None 		 None
_C2 : 8.71*m1 + 0.383141762*m2 - 114.8*n1 - 0.75*n2 - 3*n3 - 16660*n4 <= -0.0 	 None 		 None
_C3 : 8.1*m1 + 0.01618123*m2 - 106.8*n1 - 0.7*n2 - 3*n3 - 12376*n4 <= -0.0 	 None 		 None
_C4 : 6.2*m1 + 0.041407867*m2 - 84.6*n1 - 0.86*n2 - 4*n3 - 24463*n4 <= -0.0 	 None 		 None
_C5 : 3.12*m1 + 0.031152648*m2 - 77.7*n1 - 0.63*n2 - 4*n3 - 6810*n4 <= -0.0 	 None 		 None
_C6 : 0.12*m1 + 0.071428571*m2 - 56.3*n1 - 0.81*n2 - 2*n3 - 42442*n4 <= -0.0 	 None 		 None
_C7 : 1.32*m1 + 0.043956044*m2 - 45.8*n1 - 0.73*n2 - 2*n3 - 9639*n4 <= -0.0 	 None 		 None
_C8 : 3.07*m1 + 0.1953125*m2 - 37.8*n1 - 0.76*n2 - 3*n3 - 17515*n4 <= -0.0 	 None 		 None
_C9 : 

In [55]:
efficiency_table =np.empty((9,9))
average_list =[]
for row in range(9):
    row_values=[]
    for column in range(9):
      value = ((weights_matrix[0,column]*df["Y1"][row])+(weights_matrix[1,column]*df["Y2"][row]))/((weights_matrix[2,column]*df["X1"][row])+(weights_matrix[3,column]*df["X2"][row])+(weights_matrix[4,column]*df["X3"][row])+(weights_matrix[5,column]*df["X4"][row]))

      efficiency_table[row,column]=value
      row_values=row_values+[value]
    from statistics import mean
    average =mean(row_values)
    average_list =average_list+[average]

In [56]:
average_list_rounded = [ '%.2f' % elem for elem in average_list ]

In [57]:
country_list = ['Brazil','Vietnam','Chile','India','Kazakhstan','Egypt','Mexico','Indonesia','Morocco' ]

In [58]:
for country in country_list:
    index = country_list.index(country)
    print("Cross-efficiency score for",country,"=",average_list_rounded[index])

Cross-efficiency score for Brazil = 0.95
Cross-efficiency score for Vietnam = 0.53
Cross-efficiency score for Chile = 0.34
Cross-efficiency score for India = 0.35
Cross-efficiency score for Kazakhstan = 0.11
Cross-efficiency score for Egypt = 0.24
Cross-efficiency score for Mexico = 0.65
Cross-efficiency score for Indonesia = 0.72
Cross-efficiency score for Morocco = 0.32


In [59]:
display(efficiency_table)
np.set_printoptions(suppress=True, precision = 6)
print(efficiency_table)
np.savetxt('table.txt', efficiency_table, fmt='%f')

array([[1.      , 0.885964, 1.000005, 0.999998, 1.000003, 1.000001,
        0.999998, 0.655342, 0.999998],
       [0.056852, 1.      , 0.997558, 0.798366, 0.0445  , 0.55103 ,
        0.047804, 0.820407, 0.46466 ],
       [0.073602, 0.442314, 0.712399, 0.328734, 0.120427, 0.524943,
        0.117501, 0.317692, 0.427125],
       [0.198913, 0.453233, 0.458496, 0.619246, 0.095265, 0.298007,
        0.133332, 0.574291, 0.313105],
       [0.07318 , 0.006245, 0.015876, 0.031127, 0.346621, 0.182033,
        0.185565, 0.003544, 0.138999],
       [0.198291, 0.221439, 0.19769 , 0.237784, 0.24358 , 0.307465,
        0.258474, 0.171659, 0.305044],
       [0.484881, 0.301673, 0.459465, 0.392042, 1.000002, 1.000001,
        0.999999, 0.219712, 0.999998],
       [0.140089, 1.      , 1.000005, 0.999997, 0.131659, 1.000002,
        0.184954, 0.999998, 0.999998],
       [0.190985, 0.312116, 0.320501, 0.380796, 0.181064, 0.422462,
        0.27414 , 0.32563 , 0.486347]])

[[1.       0.885964 1.000005 0.999998 1.000003 1.000001 0.999998 0.655342
  0.999998]
 [0.056852 1.       0.997558 0.798366 0.0445   0.55103  0.047804 0.820407
  0.46466 ]
 [0.073602 0.442314 0.712399 0.328734 0.120427 0.524943 0.117501 0.317692
  0.427125]
 [0.198913 0.453233 0.458496 0.619246 0.095265 0.298007 0.133332 0.574291
  0.313105]
 [0.07318  0.006245 0.015876 0.031127 0.346621 0.182033 0.185565 0.003544
  0.138999]
 [0.198291 0.221439 0.19769  0.237784 0.24358  0.307465 0.258474 0.171659
  0.305044]
 [0.484881 0.301673 0.459465 0.392042 1.000002 1.000001 0.999999 0.219712
  0.999998]
 [0.140089 1.       1.000005 0.999997 0.131659 1.000002 0.184954 0.999998
  0.999998]
 [0.190985 0.312116 0.320501 0.380796 0.181064 0.422462 0.27414  0.32563
  0.486347]]
